In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col,count,max,min,avg,sum,udf
spark = SparkSession.builder.appName("OfficeDataProject").getOrCreate()
df = spark.read.options(header=True, inferSchema=True).csv("/FileStore/tables/officedataproject.csv")
df.show()

+-----------+-------------------+----------+-----+------+---+-----+
|employee_id|      employee_name|department|state|salary|age|bonus|
+-----------+-------------------+----------+-----+------+---+-----+
|       1000|          Nitz Leif| Marketing|   CA|  6131| 26|  543|
|       1001|    Melissia Dedman|   Finance|   AK|  4027| 43| 1290|
|       1002|  Rudolph Barringer|        HR|   LA|  3122| 43| 1445|
|       1003|        Tamra Amber|  Accounts|   AK|  5717| 47| 1291|
|       1004|        Mullan Nitz|Purchasing|   CA|  5685| 34| 1394|
|       1005|      Zollner Karie|  Accounts|   CA|  2843| 27| 1078|
|       1006|Kaczorowski Zollner|     Sales|   CA|  7201| 21| 1834|
|       1007|      Nakano Locust| Marketing|   LA|  3444| 23| 1823|
|       1008|  Recalde Kensinger|  Accounts|   LA|  3704| 48| 1330|
|       1009|        Imai Hallie|  Accounts|   AK|  5061| 38| 1557|
|       1010|    Debroah Gallman|  Accounts|   NY|  9308| 35|  817|
|       1011|   Barringer Escoto|Purchasing|   W

Print total number of employees in the company

In [0]:
df.select(df.employee_name).count()

Out[2]: 1000

Print total number of departments in the company

In [0]:
df.select(df.department).dropDuplicates().count()

Out[3]: 6

Print Department names of the company

In [0]:
df.select(df.department).dropDuplicates().show()

+----------+
|department|
+----------+
|     Sales|
|        HR|
|   Finance|
|Purchasing|
| Marketing|
|  Accounts|
+----------+



Print total number of employees in each department

In [0]:
df.groupBy(df.department).count().show()

+----------+-----+
|department|count|
+----------+-----+
|     Sales|  169|
|        HR|  171|
|   Finance|  162|
|Purchasing|  166|
| Marketing|  170|
|  Accounts|  162|
+----------+-----+



Print total number of employees in each state

In [0]:
df.groupBy(df.state).count().show()

+-----+-----+
|state|count|
+-----+-----+
|   LA|  205|
|   CA|  205|
|   WA|  208|
|   NY|  173|
|   AK|  209|
+-----+-----+



Print the total number of employees in each state in each department

In [0]:
df.groupBy(df.state,df.department).count().show()

+-----+----------+-----+
|state|department|count|
+-----+----------+-----+
|   CA|     Sales|   42|
|   CA| Marketing|   33|
|   NY|  Accounts|   34|
|   NY|     Sales|   27|
|   CA|   Finance|   35|
|   CA|  Accounts|   35|
|   CA|Purchasing|   32|
|   WA|        HR|   47|
|   AK|Purchasing|   30|
|   WA|  Accounts|   27|
|   WA|Purchasing|   38|
|   AK|     Sales|   38|
|   AK|  Accounts|   37|
|   WA| Marketing|   39|
|   LA|        HR|   41|
|   LA|     Sales|   35|
|   AK|        HR|   25|
|   LA|   Finance|   29|
|   AK|   Finance|   37|
|   LA|Purchasing|   45|
+-----+----------+-----+
only showing top 20 rows



Print the minimum and maximum salaries in each department and sort them in ascending order

In [0]:
df.groupBy(df.department).agg(min(df.salary),max(df.salary)).orderBy(col("min(salary)"),col("max(salary)")).show()

+----------+-----------+-----------+
|department|min(salary)|max(salary)|
+----------+-----------+-----------+
|   Finance|       1006|       9899|
|  Accounts|       1007|       9890|
|        HR|       1013|       9982|
| Marketing|       1031|       9974|
|     Sales|       1103|       9982|
|Purchasing|       1105|       9985|
+----------+-----------+-----------+



Print the names of employees working in NY state under finance department whose bonuses are greater than the average bonuses of employees in NY state

In [0]:
avg_ny_state_bonus = df.filter((df.state == 'NY')).groupBy(df.state).agg(avg(df.bonus)).rdd.collect()[0]['avg(bonus)']
df.select(df.employee_name).filter((df.state == 'NY') & (df.department == 'Finance') & (df.bonus > avg_ny_state_bonus)).show()

+--------------------+
|       employee_name|
+--------------------+
|       Vivan Sifford|
|      Herder Gallman|
|          Nena Rocha|
|       Leif Lemaster|
|Ellingsworth Meli...|
|        Escoto Gilma|
|     Georgeanna Laub|
|     Durio Tenenbaum|
|       Juliana Grigg|
|        Tiffani Benz|
|          Nitz Ilana|
|   Phylicia Antonina|
|         Durio Janey|
|       Melissia Jere|
|      Yukiko Kreamer|
|      Nena Kensinger|
|      Antonina Ilana|
+--------------------+



Raise the salaries $500 of all employees whose age is greater than 45

In [0]:
def increase_salary(age,salary):
    return salary+500 if age > 45 else salary

increase_salary_udf = udf(lambda x,y: increase_salary(x,y), IntegerType())
df2 = df.withColumn("salary",increase_salary_udf(df.age,df.salary))
df2.show()

+-----------+-------------------+----------+-----+------+---+-----+
|employee_id|      employee_name|department|state|salary|age|bonus|
+-----------+-------------------+----------+-----+------+---+-----+
|       1000|          Nitz Leif| Marketing|   CA|  6131| 26|  543|
|       1001|    Melissia Dedman|   Finance|   AK|  4027| 43| 1290|
|       1002|  Rudolph Barringer|        HR|   LA|  3122| 43| 1445|
|       1003|        Tamra Amber|  Accounts|   AK|  6217| 47| 1291|
|       1004|        Mullan Nitz|Purchasing|   CA|  5685| 34| 1394|
|       1005|      Zollner Karie|  Accounts|   CA|  2843| 27| 1078|
|       1006|Kaczorowski Zollner|     Sales|   CA|  7201| 21| 1834|
|       1007|      Nakano Locust| Marketing|   LA|  3444| 23| 1823|
|       1008|  Recalde Kensinger|  Accounts|   LA|  4204| 48| 1330|
|       1009|        Imai Hallie|  Accounts|   AK|  5061| 38| 1557|
|       1010|    Debroah Gallman|  Accounts|   NY|  9308| 35|  817|
|       1011|   Barringer Escoto|Purchasing|   W